# 4. Machine Learning Models

- **Modelos ML:** Vale a pena mencionar o facto de o seguinte conjunto de dados ser altamente desequilibrado. Por isso, criámos um conjunto de dados equilibrado a partir do nosso conhecimento do domínio para treinar vários modelos ML. Uma vez que o nosso conjunto de dados é bastante grande e tem uma quantidade razoável de amostras para treinar e testar diferentes modelos de ML utilizando vários algoritmos de classificação (Regressão logística, Máquina de vectores de apoio para classificação binária e Classificador de floresta aleatória, Árvore de decisão, K vizinhos mais próximos para classificação multi-classe).

  Para classificações binárias, treinámos os modelos para distinguir entre tráfego normal e tráfego anómalo. Isto significa que apenas prevê se está a ocorrer uma intrusão ou não. Em alternativa, utilizando os algoritmos de classificação multi-classe, alargámos ainda mais as nossas capacidades de previsão para identificar o tipo de ataque ou intrusão que está a ocorrer. Experimentámos classificações binárias e classificações multi-classe para ver como os dados se comportam. Posteriormente, efectuámos uma validação cruzada, avaliámos e comparámos esses modelos para ver qual deles funciona melhor ou pior.

In [1]:
# Importando bibliotecas principais
import pandas as pd  # Biblioteca principal para manipulação de dados
import numpy as np  # Biblioteca para manipulação numérica, como arrays e operações com matrizes

# Importando ferramentas para visualizações
import seaborn as sns  # Para visualizações de dados, como heatmaps
import matplotlib.pyplot as plt  # Para visualizações, como gráficos
from sklearn.decomposition import IncrementalPCA  # Para redução de dimensionalidade com PCA incremental
from sklearn.preprocessing import StandardScaler  # Para normalizar os dados
from sklearn.model_selection import train_test_split  # Para dividir os dados em treino e teste
from sklearn.ensemble import RandomForestClassifier  # Algoritmo de aprendizado de máquina
from sklearn.metrics import classification_report, confusion_matrix  # Métricas de avaliação do modelo

In [2]:
new_data = pd.read_csv('datasets/CICIDS2017/processed_dataset.csv')

In [3]:
new_data.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC27,PC28,PC29,PC30,PC31,PC32,PC33,PC34,PC35,Attack Types
0,-2.311094,-0.052684,0.515875,0.616537,3.840727,0.395336,-0.017878,0.186557,0.370079,-0.680438,...,-0.233945,0.699887,-0.539784,-0.035279,0.023075,0.001733,0.045647,0.151301,0.051935,BENIGN
1,-2.246553,-0.049159,0.467881,0.395550,2.001551,-0.141045,-0.016487,-0.780967,-0.889976,2.660582,...,-0.012903,0.543839,0.785574,0.212940,0.030939,0.001218,0.025860,0.009198,-0.058324,BENIGN
2,-2.258822,-0.049501,0.473634,0.408672,2.081408,-0.132962,-0.016754,-0.769681,-0.877466,2.634068,...,-0.020362,0.539934,0.780933,0.203564,0.034717,0.001194,0.025898,0.005914,-0.064384,BENIGN
3,-2.249188,-0.050635,0.467051,0.346824,2.013841,-0.106530,-0.016178,-0.745133,-0.840229,2.506786,...,-0.027380,0.175826,0.802770,0.063856,0.047573,0.001131,0.009281,0.018965,-0.033301,BENIGN
4,-2.311090,-0.052685,0.515877,0.616515,3.840698,0.395327,-0.017879,0.186543,0.370100,-0.680454,...,-0.233960,0.699809,-0.539762,-0.035319,0.023078,0.001733,0.045645,0.151311,0.051946,BENIGN


In [4]:
# Para cross validation
from sklearn.model_selection import cross_val_score

## 4.1. Criando um conjunto de dados balanceado para classificação binária

Sabemos que um conjunto de dados equilibrado é crucial na aprendizagem automática porque garante que cada classe ou categoria de dados é representada de forma igual. Isto significa que o número de observações em cada classe é aproximadamente o mesmo, o que evita que o modelo seja enviesado para a classe maioritária. Um conjunto de dados enviesado pode levar a um fraco desempenho do modelo, uma vez que este pode ter dificuldade em prever as classes minoritárias. Como já sabemos que o seguinte conjunto de dados é altamente desequilibrado, recorremos à ajuda da SMOTE (Synthetic Minority Over-sampling Technique) para aumentar a amostragem das classes minoritárias enquanto criamos um conjunto de dados equilibrado para a classificação multiclasse. Isto ajudou-nos a criar um conjunto de dados globalmente equilibrado para alimentar os modelos de classificação.

In [5]:
# Separando o tráfego normal (BENIGN) do tráfego de intrusão (outros tipos de ataque)
normal_traffic = new_data.loc[new_data['Attack Types'] == 'BENIGN']  # Seleciona todas as linhas onde o ataque é BENIGN (tráfego normal)
intrusions = new_data.loc[new_data['Attack Types'] != 'BENIGN']  # Seleciona todas as linhas onde o ataque é diferente de BENIGN (intrusões)

# Amostrando o mesmo número de exemplos de tráfego normal para igualar ao número de intrusões
normal_traffic = normal_traffic.sample(n=len(intrusions), replace=False)  # Pega uma amostra aleatória do tráfego normal com o mesmo tamanho do tráfego de intrusões

# Combinando as amostras de intrusão e tráfego normal em um novo DataFrame balanceado
ids_data = pd.concat([intrusions, normal_traffic])  # Concatena as duas amostras para formar um dataset balanceado

# Convertendo a coluna 'Attack Type' em uma variável binária:
# 0 para tráfego normal (BENIGN) e 1 para intrusão (outros ataques)
ids_data['Attack Types'] = np.where((ids_data['Attack Types'] == 'BENIGN'), 0, 1)

In [6]:
# Verificando o número total de registros no dataset balanceado
print(f'Tamanho do dataset balanceado: {ids_data.shape[0]} registros')

# Contando a quantidade de registros de cada classe (0 = BENIGN, 1 = Intrusão)
print('\nDistribuição das classes:')
print(ids_data['Attack Types'].value_counts())


Tamanho do dataset balanceado: 851756 registros

Distribuição das classes:
Attack Types
1    425878
0    425878
Name: count, dtype: int64


In [7]:
# Criando uma amostra final de 15.000 registros para análise ou treinamento
bc_data = ids_data.sample(n=300000)  # Reduz o dataset balanceado para 15.000 amostras, escolhendo aleatoriamente

# Imprimindo a contagem de classes para verificar o balanceamento final
print(bc_data['Attack Types'].value_counts())  # Exibe a quantidade de registros para cada classe (0 ou 1) no dataset final

Attack Types
0    150084
1    149916
Name: count, dtype: int64


In [8]:
# Importando a função train_test_split do módulo sklearn.model_selection
from sklearn.model_selection import train_test_split

# Separando os dados em características (X) e a variável alvo (y)
X_bc = bc_data.drop('Attack Types', axis=1)  # 'X_bc' contém todas as colunas, exceto 'Attack Type'
y_bc = bc_data['Attack Types']  # 'y_bc' contém apenas a coluna 'Attack Type', que é a variável alvo

# Dividindo os dados em conjuntos de treinamento e teste
X_train_bc, X_test_bc, y_train_bc, y_test_bc = train_test_split(
    X_bc,              # Dados de entrada (features)
    y_bc,              # Variável alvo (target)
    test_size=0.25,    # 25% dos dados serão usados para o conjunto de teste, 75% para treinamento
    random_state=0     # Garante que a divisão dos dados seja reproduzível (mesma divisão a cada execução)
)


In [9]:
X_bc.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33,PC34,PC35
219977,-1.920537,-0.037763,0.205313,-0.220464,0.573357,0.555330,0.033784,-0.047656,-0.692434,0.295734,...,0.522395,-0.154476,-0.677817,-0.986973,-0.432997,0.094646,0.001562,-0.023793,0.210730,0.205771
335488,-1.942302,0.003643,-0.118037,-0.210338,-0.442203,0.665356,0.071809,0.314913,-0.930998,-0.241096,...,-1.265018,-0.537686,0.525493,0.456290,0.296651,-0.245101,0.001003,-0.018109,0.203309,0.329215
1363677,-2.247349,-0.021353,0.309430,0.484723,1.904217,0.776808,0.070743,-0.228512,-1.941442,2.540625,...,-0.929542,0.038544,0.616835,-2.059240,0.051088,0.135748,-0.001106,0.141669,0.038056,0.076470
403863,-1.920943,0.002792,-0.130051,-0.285585,-0.585648,0.683850,0.072563,0.328974,-0.903561,-0.346551,...,-1.317450,-0.537965,0.161667,0.483445,0.164359,-0.235846,0.000962,-0.035035,0.219783,0.366303
2135483,16.422763,-0.450907,-2.528852,-2.491848,1.032512,-1.155682,-0.115230,-2.307312,-0.508401,-2.191145,...,1.374749,-1.621366,-1.456983,-0.299889,2.158799,-1.621026,-0.000169,-0.050519,-0.082597,0.029701


### 4.2 Testando diferentes Modelos

#### 4.2.1 Regressão logística (classificação binária)

A regressão logística é um tipo de modelo estatístico utilizado para prever a probabilidade de um resultado binário com base numa ou mais variáveis independentes. Modela a relação entre a variável independente e a variável dependente utilizando uma função sigmoide para produzir uma pontuação de probabilidade entre 0 e 1. É frequentemente utilizada em tarefas de classificação em que o objetivo é determinar a qual das duas classes pertence uma observação, como, por exemplo, se um e-mail é spam ou não.

Parâmetros:
max_iter: este parâmetro define o número máximo de iterações para o solucionador convergir. O valor por defeito é definido como 100. No entanto, o nosso modelo não conseguiu convergir com apenas 100 iterações, pelo que o aumentámos para o valor desejado.

C: Este parâmetro é a força de regularização e controla o compromisso entre ajustar bem os dados de treino e evitar o sobreajuste. Um valor mais pequeno de C especifica uma regularização mais forte. Utilizámos um valor mais baixo para um modelo e um valor mais alto para o outro para ver o desempenho dos modelos em termos de evitar o sobreajuste depois de atribuir uma importância elevada e baixa, respetivamente.

solver: Este parâmetro especifica o algoritmo a utilizar no problema de otimização quando se ajusta o modelo de regressão logística. Existem vários algoritmos de resolução diferentes disponíveis, tais como lbfgs, saga, liblinear e alguns outros. Optámos por 'saga' e 'sag' para treinar os nossos modelos.

random_state: Isto é para garantir que o resultado é determinístico e pode ser reproduzido.

In [10]:
# Importando a classe LogisticRegression do módulo sklearn.linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score  # Para validação cruzada

# Criando uma instância do modelo de Regressão Logística com parâmetros específicos
lr1 = LogisticRegression(
    max_iter=10000,      # Define o número máximo de iterações para a convergência do algoritmo
    C=0.1,               # Inverso da força de regularização (menores valores = maior regularização)
    random_state=0,      # Garante que os resultados sejam reproduzíveis
    solver='saga'        # Algoritmo de otimização 'saga' adequado para grandes datasets e regularização
)

# Treinando o modelo de Regressão Logística com o conjunto de treinamento
lr1.fit(X_train_bc, y_train_bc)

# Realizando a validação cruzada (cross-validation) com 5 divisões (folds) no conjunto de treinamento
cv_lr1 = cross_val_score(lr1, X_train_bc, y_train_bc, cv=5)

# Exibindo os resultados da validação cruzada
print('Logistic Regression Model 1')
print(f'\nCross-validation scores:', ', '.join(map(str, cv_lr1)))  # Imprime os escores individuais
print(f'\nMean cross-validation score: {cv_lr1.mean():.2f}')       # Imprime a média dos escores

Logistic Regression Model 1

Cross-validation scores: 0.9320888888888889, 0.9321333333333334, 0.9324, 0.9339777777777778, 0.9310888888888889

Mean cross-validation score: 0.93


In [11]:
# Exibindo os coeficientes do modelo de Regressão Logística
print('Logistic Regression Model 1 coefficients:')
print(*lr1.coef_, sep=', ')  # Desempacota os coeficientes para exibir cada um separadamente

# Exibindo o intercepto do modelo (termo constante)
print('\nLogistic Regression Model 1 intercept:', *lr1.intercept_)

Logistic Regression Model 1 coefficients:
[ 0.61481106 -0.28782001 -0.73968468 -2.22895372  0.93182368  0.84528959
  0.00529187  0.50796788 -0.06020218  0.42085667  0.09252799 -0.08415574
 -0.06952727  0.53110525  0.72171644  0.10740986 -0.02412979 -0.81313946
 -0.29807555  0.61103451 -0.43693979 -0.6477885   0.58660052  2.43578232
 -1.43815042 -1.69212272 -2.41827288 -0.51192494 -0.78447961  2.17524913
  0.0392921   0.01312341 -4.37151276 -1.26864002  4.04246499]

Logistic Regression Model 1 intercept: -3.453152657918483


In [12]:
# Criando uma segunda instância do modelo de Regressão Logística com diferentes parâmetros
lr2 = LogisticRegression(
    max_iter=15000,      # Aumenta o número máximo de iterações para garantir a convergência
    solver='sag',        # Utiliza o solver 'sag', eficiente para grandes conjuntos de dados
    C=100,               # Menor regularização (C alto significa menos penalização)
    random_state=0       # Garante reprodutibilidade dos resultados
)

# Treinando o modelo de Regressão Logística com o conjunto de treinamento
lr2.fit(X_train_bc, y_train_bc)

# Realizando validação cruzada (cross-validation) com 5 divisões no conjunto de treinamento
cv_lr2 = cross_val_score(lr2, X_train_bc, y_train_bc, cv=5)

# Exibindo os resultados da validação cruzada
print('Logistic Regression Model 2')
print(f'\nCross-validation scores:', ', '.join(map(str, cv_lr2)))  # Imprime os scores individuais de cada fold
print(f'\nMean cross-validation score: {cv_lr2.mean():.2f}')       # Imprime a média dos scores

Logistic Regression Model 2

Cross-validation scores: 0.934, 0.9336, 0.934, 0.9349777777777778, 0.9327555555555556

Mean cross-validation score: 0.93


In [13]:
# Exibindo os coeficientes do Modelo 2 de Regressão Logística
print('Logistic Regression Model 2 coefficients:')
print(*lr2.coef_, sep=', ')  # Desempacota a lista de coeficientes para exibir de forma clara e separada

# Exibindo o intercepto do Modelo 2
print('\nLogistic Regression Model 2 intercept:', *lr2.intercept_)  # Mostra o termo constante do modelo

Logistic Regression Model 2 coefficients:
[ 6.67985800e-01 -2.74008973e-01 -8.16972578e-01 -2.36073192e+00
  1.05044777e+00  8.84889278e-01  3.27564643e-03  5.80710870e-01
 -7.77714846e-02  4.93411230e-01  1.31633835e-01 -1.31323516e-01
 -3.69345898e-02  5.56126558e-01  8.13215697e-01  1.03925814e-01
 -3.18371241e-02 -8.62290917e-01 -3.74177353e-01  6.46039756e-01
 -4.77008705e-01 -6.81531246e-01  5.61710459e-01  2.55889912e+00
 -1.63895472e+00 -1.85884196e+00 -2.69932883e+00 -6.65972245e-01
 -8.71472732e-01  2.53713361e+00  1.44822654e-01  1.60122661e-02
 -4.93943328e+00 -2.01156298e+00  4.31450119e+00]

Logistic Regression Model 2 intercept: -3.629924077835758


#### 4.2.2 Máquina de Vectores de Suporte (Classificação Binária)

A máquina de vectores de suporte (SVM) é um tipo de algoritmo de aprendizagem automática supervisionada utilizado para a análise de classificação e regressão. Funciona encontrando um hiperplano num espaço de elevada dimensão que melhor separa os pontos de dados em diferentes classes.

Parâmetros:
kernel: O parâmetro kernel especifica o tipo de função kernel a utilizar. Neste caso, utilizámos rbf e poly kernel.

C: O parâmetro C controla o compromisso entre a maximização da margem e a minimização do erro de classificação.

gamma: O parâmetro gamma é um hiperparâmetro que determina a influência de um único exemplo de treinamento no limite de decisão.

random_state: Este parâmetro serve para garantir que o resultado seja determinístico e possa ser reproduzido.

In [ ]:
# Importando o modelo de Support Vector Classification (SVC)
from sklearn.svm import SVC

# Criando uma instância do modelo SVM com um kernel polinomial
svm1 = SVC(
    kernel='poly',        # Utiliza um kernel polinomial para transformar os dados em um espaço de maior dimensão
    C=1,                  # Parâmetro de regularização: controla o equilíbrio entre acerto e generalização
    random_state=0,       # Garante reprodutibilidade dos resultados
    probability=True      # Ativa a estimativa de probabilidades (necessário para algumas métricas e análises)
)

# Treinando o modelo SVM com o conjunto de treinamento
svm1.fit(X_train_bc, y_train_bc)

# Realizando validação cruzada com 5 divisões (folds)
cv_svm1 = cross_val_score(svm1, X_train_bc, y_train_bc, cv=5)

# Exibindo os resultados da validação cruzada
print('Support Vector Machine Model 1')
print(f'\nCross-validation scores:', ', '.join(map(str, cv_svm1)))   # Imprime as pontuações individuais de cada fold
print(f'\nMean cross-validation score: {cv_svm1.mean():.2f}')         # Exibe a média das pontuações da validação cruzada

In [ ]:
# Criando uma instância do modelo SVM com kernel RBF (Radial Basis Function)
svm2 = SVC(
    kernel='rbf',         # O kernel RBF (Radial Basis Function) é utilizado para capturar relações não lineares entre as classes
    C=1,                  # Parâmetro de regularização. Um valor maior de C tenta ajustar o modelo o mais possível aos dados de treino, 
                         # mas pode resultar em overfitting se for grande demais.
    gamma=0.1,            # Parâmetro que controla a forma da função kernel. Um valor pequeno de gamma implica que cada ponto de treino
                         # tem uma influência maior no modelo, enquanto um valor grande pode resultar em um modelo muito complexo.
    random_state=0,       # Garante a reprodutibilidade do modelo com o mesmo valor de semente aleatória.
    probability=True      # Habilita o cálculo de probabilidades de previsão, necessário para algumas métricas como log-loss.
)

# Treinando o modelo SVM com o conjunto de treino (X_train_bc, y_train_bc)
svm2.fit(X_train_bc, y_train_bc)

# Realizando a validação cruzada (5-fold CV) para avaliar o desempenho do modelo
cv_svm2 = cross_val_score(svm2, X_train_bc, y_train_bc, cv=5)

# Exibindo os resultados da validação cruzada
print('Support Vector Machine Model 2')
print(f'\nCross-validation scores:', ', '.join(map(str, cv_svm2)))   # Exibe as pontuações de cada uma das 5 divisões (folds)
print(f'\nMean cross-validation score: {cv_svm2.mean():.2f}')         # Exibe a média das pontuações de todos os folds

In [ ]:
# Exibe o intercepto (viés) do modelo SVM 1
print('SVM Model 1 intercept:', *svm1.intercept_)

# Exibe o intercepto (viés) do modelo SVM 2
print('SVM Model 2 intercept:', *svm2.intercept_)

## 4.2. Criação de um conjunto de dados equilibrado para classificação multi-classe

In [ ]:
new_data['Attack Types'].value_counts()

In [ ]:
# Conta o número de ocorrências de cada classe (valor único na coluna 'Attack Type')
class_counts = new_data['Attack Types'].value_counts()

# Seleciona as classes que têm mais do que 1950 registros
selected_classes = class_counts[class_counts > 1950]

# Extrai os nomes das classes selecionadas
class_names = selected_classes.index

# Cria um novo DataFrame contendo apenas as classes selecionadas
selected = new_data[new_data['Attack Types'].isin(class_names)]

# Cria uma lista para armazenar os DataFrames de cada classe selecionada
dfs = []

# Para cada classe selecionada
for name in class_names:
  # Filtra o DataFrame para a classe atual
  df = selected[selected['Attack Types'] == name]
  
  # Se o número de registros para essa classe for maior do que 2500, seleciona aleatoriamente 5000 registros
  if len(df) > 2500:
    df = df.sample(n = 5000, random_state = 0)

  # Adiciona o DataFrame da classe à lista
  dfs.append(df)

# Concatena todos os DataFrames da lista em um único DataFrame, ignorando os índices
df = pd.concat(dfs, ignore_index = True)

# Exibe a contagem de registros de cada classe no DataFrame resultante
df['Attack Types'].value_counts()


método SMOTE (Synthetic Minority Over-sampling Technique) para balancear o dataset, criando novas amostras sintéticas das classes minoritárias.

In [ ]:
from imblearn.over_sampling import SMOTE

# X contém as features (características), e y contém os rótulos (Target 'Attack Type')
X = df.drop('Attack Types', axis=1)  # Remove a coluna 'Attack Type' para ficar com as features
y = df['Attack Types']  # A coluna 'Attack Type' é o target (variável dependente)

# Inicializando o SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=0)

# Realiza o reamostramento usando o SMOTE, gerando amostras sintéticas
X_upsampled, y_upsampled = smote.fit_resample(X, y)

# Concatena as novas amostras sintéticas de volta em um DataFrame
blnc_data = pd.DataFrame(X_upsampled)

# Adiciona a coluna 'Attack Type' ao DataFrame das amostras sintéticas
blnc_data['Attack Types'] = y_upsampled

# Embaralha os dados para garantir que a distribuição das classes esteja misturada
blnc_data = blnc_data.sample(frac=1)

# Exibe a distribuição das classes no dataset balanceado
blnc_data['Attack Types'].value_counts()

In [ ]:
# Separando features (variáveis independentes) e labels (variável dependente)
features = blnc_data.drop('Attack Types', axis = 1)
labels = blnc_data['Attack Types']

# Dividindo os dados em treino e teste (75% treino, 25% teste)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.25, random_state = 0)

# Verificando o tamanho dos dados
print(f'Tamanho dos dados de treino: {X_train.shape[0]}')
print(f'Tamanho dos dados de teste: {X_test.shape[0]}')

#### Random Forest Classifier

A floresta aleatória é um método de aprendizagem de conjunto que combina várias árvores de decisão para melhorar a precisão e o desempenho de generalização do modelo. A ideia básica por trás das florestas aleatórias é ajustar várias árvores de decisão em subconjuntos aleatórios dos dados de treino e calcular a média das suas previsões para reduzir o sobreajuste e melhorar o desempenho da generalização.

Parâmetros:
n_estimadores: Este parâmetro especifica o número de árvores de decisão a serem ajustadas na floresta aleatória.

max_depth: Este parâmetro especifica a profundidade máxima de cada árvore de decisão na floresta aleatória. Uma árvore mais profunda pode capturar interações mais complexas nos dados. No nosso caso, este parâmetro desempenhou um papel importante na obtenção de melhores resultados.

max_features: Este parâmetro especifica o número de caraterísticas a considerar ao procurar a melhor divisão em cada árvore. Treinámos o primeiro modelo tendo em conta todas as caraterísticas e, para o segundo, utilizámos apenas 20 caraterísticas.

random_state: Como mencionado anteriormente, este parâmetro serve para garantir que o resultado é determinístico e pode ser reproduzido.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Criando o modelo de Random Forest com parâmetros específicos
rf1 = RandomForestClassifier(n_estimators = 10, max_depth = 6, max_features = None, random_state = 0)

# Treinando o modelo com os dados de treino
rf1.fit(X_train, y_train)

# Realizando a validação cruzada com 5 divisões (folds)
cv_rf1 = cross_val_score(rf1, X_train, y_train, cv = 5)

# Exibindo os resultados da validação cruzada
print('Random Forest Model 1')
print(f'\nCross-validation scores:', ', '.join(map(str, cv_rf1)))  # Mostra as pontuações de cada fold
print(f'\nMean cross-validation score: {cv_rf1.mean():.2f}')  # Exibe a média das pontuações de validação cruzada

In [ ]:
# Criando o modelo de Random Forest com parâmetros específicos
rf2 = RandomForestClassifier(n_estimators = 15, max_depth = 8, max_features = 20, random_state = 0)

# Treinando o modelo com os dados de treino
rf2.fit(X_train, y_train)

# Realizando a validação cruzada com 5 divisões (folds)
cv_rf2 = cross_val_score(rf2, X_train, y_train, cv = 5)

# Exibindo os resultados da validação cruzada
print('Random Forest Model 2')
print(f'\nCross-validation scores:', ', '.join(map(str, cv_rf2)))  # Mostra as pontuações de cada fold
print(f'\nMean cross-validation score: {cv_rf2.mean():.2f}')  # Exibe a média das pontuações de validação cruzada

#### Decision Tree Classifier

Uma árvore de decisão é um tipo de algoritmo utilizado na aprendizagem automática para tarefas de classificação e regressão. O algoritmo funciona dividindo recursivamente os dados em subconjuntos mais pequenos, com base nos valores das caraterísticas de entrada, até ser cumprido um critério de paragem. No nosso caso, é a profundidade máxima da árvore.

Parâmetros:
max_depth: Este parâmetro especifica a profundidade máxima da árvore. Uma árvore mais profunda pode capturar interações mais complexas nos dados, mas pode ser computacionalmente dispendiosa. Começámos com uma profundidade pequena e depois aumentámo-la.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

# Criando o modelo de Decision Tree com profundidade máxima de 6
dt1 = DecisionTreeClassifier(max_depth = 6)

# Treinando o modelo com os dados de treino
dt1.fit(X_train, y_train)

# Realizando a validação cruzada com 5 divisões (folds)
cv_dt1 = cross_val_score(dt1, X_train, y_train, cv = 5)

# Exibindo os resultados da validação cruzada
print('Decision Tree Model 1')
print(f'\nCross-validation scores:', ', '.join(map(str, cv_dt1)))  # Mostra as pontuações de cada fold
print(f'\nMean cross-validation score: {cv_dt1.mean():.2f}')  # Exibe a média das pontuações de validação cruzada

In [ ]:
# Criando o modelo de Decision Tree com profundidade máxima de 8
dt2 = DecisionTreeClassifier(max_depth = 8)

# Treinando o modelo com os dados de treino
dt2.fit(X_train, y_train)

# Realizando a validação cruzada com 5 divisões (folds)
cv_dt2 = cross_val_score(dt2, X_train, y_train, cv = 5)

# Exibindo os resultados da validação cruzada
print('Decision Tree Model 2')
print(f'\nCross-validation scores:', ', '.join(map(str, cv_dt2)))  # Mostra as pontuações de cada fold
print(f'\nMean cross-validation score: {cv_dt2.mean():.2f}')  # Exibe a média das pontuações de validação cruzada

#### K Nearest Neighbours

K Nearest Neighbors (KNN) é um algoritmo simples que procura os k pontos de dados mais próximos (vizinhos) no conjunto de treino para o novo ponto de dados de entrada, com base numa métrica de distância, normalmente a distância euclidiana. Em seguida, o algoritmo adopta um voto maioritário para a classificação das etiquetas ou valores-alvo desses k vizinhos para prever a etiqueta ou o valor-alvo do novo ponto de dados.

Parâmetros:
n_vizinhos: Este é um hiperparâmetro do algoritmo KNN que especifica o número de vizinhos a considerar ao fazer previsões para um novo ponto de dados de entrada. No nosso caso, começámos inicialmente com 16 para fazer previsões. Assim, o modelo considerará os 16 pontos de dados mais próximos (vizinhos).

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# Criando o modelo de K-Nearest Neighbors com 16 vizinhos
knn1 = KNeighborsClassifier(n_neighbors = 16)

# Treinando o modelo com os dados de treino
knn1.fit(X_train, y_train)

# Realizando a validação cruzada com 5 divisões (folds)
cv_knn1 = cross_val_score(knn1, X_train, y_train, cv = 5)

# Exibindo os resultados da validação cruzada
print('K Nearest Neighbors Model 1')
print(f'\nCross-validation scores:', ', '.join(map(str, cv_knn1)))  # Mostra as pontuações de cada fold
print(f'\nMean cross-validation score: {cv_knn1.mean():.2f}')  # Exibe a média das pontuações de validação cruzada

In [ ]:
# Criando o modelo de K-Nearest Neighbors com 8 vizinhos
knn2 = KNeighborsClassifier(n_neighbors = 8)

# Treinando o modelo com os dados de treino
knn2.fit(X_train, y_train)

# Realizando a validação cruzada com 5 divisões (folds)
cv_knn2 = cross_val_score(knn2, X_train, y_train, cv = 5)

# Exibindo os resultados da validação cruzada
print('K Nearest Neighbors Model 2')
print(f'\nCross-validation scores:', ', '.join(map(str, cv_knn2)))  # Mostra as pontuações de cada fold
print(f'\nMean cross-validation score: {cv_knn2.mean():.2f}')  # Exibe a média das pontuações de validação cruzada

# 5. Avaliação e discussão do desempenho

In [ ]:
## Importing necessary functions
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, classification_report, \
 roc_auc_score, roc_curve, auc, precision_recall_curve

### Comparação de modelos de regressão logística

In [ ]:
y_pred_lr1 = lr1.predict(X_test_bc)
y_pred_lr2 = lr2.predict(X_test_bc)

conf_matrix_model1 = confusion_matrix(y_test_bc, y_pred_lr1)
conf_matrix_model2 = confusion_matrix(y_test_bc, y_pred_lr2)

fig, axs = plt.subplots(1, 2, figsize = (12, 4))

sns.heatmap(conf_matrix_model1, annot = True, cmap = 'Blues', ax = axs[0])
axs[0].set_title('Model 1')

sns.heatmap(conf_matrix_model2, annot = True, cmap = 'Blues', ax = axs[1])
axs[1].set_title('Model 2')

axs[0].set_xlabel('Predicted label')
axs[0].set_ylabel('True label')
axs[1].set_xlabel('Predicted label')
plt.show()

In [ ]:
y_prob_lr1 = lr1.predict_proba(X_test_bc)[:,1]
y_prob_lr2 = lr2.predict_proba(X_test_bc)[:,1]

fpr1, tpr1, _ = roc_curve(y_test_bc, y_prob_lr1)
roc_auc1 = auc(fpr1, tpr1)

fpr2, tpr2, _ = roc_curve(y_test_bc, y_prob_lr2)
roc_auc2 = auc(fpr2, tpr2)

colors = sns.color_palette('Set2', n_colors = 3)
fig, axes = plt.subplots(1, 3, figsize = (15, 5))

axes[0].plot(fpr1, tpr1, label = f'ROC curve (area = {roc_auc1:.2%})', color = colors[1])
axes[0].plot([0, 1], [0, 1], color = colors[0], linestyle = '--')
axes[0].set_xlim([-0.05, 1.0])
axes[0].set_ylim([0.0, 1.05])
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate')
axes[0].set_title('ROC Curve (Model 1)')
axes[0].legend(loc = 'lower right')

axes[1].plot(fpr2, tpr2, label = f'ROC curve (area = {roc_auc2:.2%})', color = colors[2])
axes[1].plot([0, 1], [0, 1], color = colors[0], linestyle = '--')
axes[1].set_xlim([-0.05, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('ROC Curve (Model 2)')
axes[1].legend(loc = 'lower right')

axes[2].plot(fpr1, tpr1, label = f'ROC curve (area = {roc_auc1:.2%})', color = colors[1])
axes[2].plot(fpr2, tpr2, label = f'ROC curve (area = {roc_auc2:.2%})', color = colors[2])
axes[2].plot([0, 1], [0, 1], color = colors[0], linestyle = '--')
axes[2].set_xlim([-0.05, 1.0])
axes[2].set_ylim([0.0, 1.05])
axes[2].set_xlabel('False Positive Rate')
axes[2].set_ylabel('True Positive Rate')
axes[2].set_title('Model 1 vs Model 2')
axes[2].legend(loc = 'lower right')

plt.tight_layout()
plt.show()

In [ ]:
precision1, recall1, threshold1 = precision_recall_curve(y_test_bc, y_prob_lr1)
precision2, recall2, threshold2 = precision_recall_curve(y_test_bc, y_prob_lr2)

fig, axs = plt.subplots(1, 3, figsize = (15, 5))

axs[0].plot(recall1, precision1, color = colors[1], label = 'Model 1')
axs[0].set_xlabel('Recall')
axs[0].set_ylabel('Precision')
axs[0].set_title('Precision-Recall Curve (Model 1)')

axs[1].plot(recall2, precision2, color = colors[2], label = 'Model 2')
axs[1].set_xlabel('Recall')
axs[1].set_ylabel('Precision')
axs[1].set_title('Precision-Recall Curve (Model 2)')

axs[2].plot(recall1, precision1, color = colors[1], label = 'Model 1')
axs[2].plot(recall2, precision2, color = colors[2], label = 'Model 2')
axs[2].set_xlabel('Recall')
axs[2].set_ylabel('Precision')
axs[2].set_title('Model 1 vs Model 2')
axs[2].legend(loc = 'lower left')

plt.tight_layout()
plt.show()

In [ ]:
target_names = lr1.classes_
metrics1 = classification_report(y_true = y_test_bc, y_pred = y_pred_lr1, target_names = target_names, output_dict = True)
precision1 = [metrics1[target_name]['precision'] for target_name in target_names]
recall1 = [metrics1[target_name]['recall'] for target_name in target_names]
f1_score1 = [metrics1[target_name]['f1-score'] for target_name in target_names]

metrics2 = classification_report(y_true = y_test_bc, y_pred = y_pred_lr2, target_names = target_names, output_dict = True)
precision2 = [metrics2[target_name]['precision'] for target_name in target_names]
recall2 = [metrics2[target_name]['recall'] for target_name in target_names]
f1_score2 = [metrics2[target_name]['f1-score'] for target_name in target_names]

data1 = np.array([precision1, recall1, f1_score1])
data2 = np.array([precision2, recall2, f1_score2])
rows = ['Precision', 'Recall', 'F1-score']

fig, axs = plt.subplots(1, 2, figsize = (14, 6))
sns.heatmap(data1, cmap='Pastel1', annot = True, fmt='.2f', xticklabels = target_names, yticklabels = rows, ax = axs[0])
sns.heatmap(data2, cmap = 'Pastel1', annot = True, fmt = '.2f', xticklabels = target_names, yticklabels = rows, ax = axs[1])
axs[0].set_title('Classification Report (Model 1)')
axs[1].set_title('Classification Report (Model 2)')
plt.show()

In [ ]:
palette = sns.color_palette('Blues', n_colors = 3)

acc1 = accuracy_score(y_pred_lr1, y_test_bc)
acc2 = accuracy_score(y_pred_lr2, y_test_bc)

labels = ['Model 1', 'Model 2']
scores = [acc1, acc2]

fig, ax = plt.subplots(figsize = (9, 3))
ax.barh(labels, scores, color = palette)
ax.set_xlim([0, 1])
ax.set_xlabel('Accuracy Score')
ax.set_title('Logistic Regression Model Comparison')

for i, v in enumerate(scores):
    ax.text(v + 0.01, i, str(round(v, 3)), ha = 'left', va = 'center')

plt.show()

In [ ]:
palette = sns.color_palette('Greens', n_colors = 3)

labels = ['Model 1', 'Model 2']
scores = [cv_lr1.mean(), cv_lr2.mean()]

fig, ax = plt.subplots(figsize = (9, 3))
ax.barh(labels, scores, color = palette)
ax.set_xlim([0, 1])
ax.set_xlabel('Cross Validation Score')
ax.set_title('Logistic Regression Model Comparison (Cross Validation)')

for i, v in enumerate(scores):
    ax.text(v + 0.01, i, str(round(v, 3)), ha = 'left', va = 'center')

plt.show()

### Comparação de modelos de SVM

In [ ]:
y_pred_svm1 = svm1.predict(X_test_bc)
y_pred_svm2 = svm2.predict(X_test_bc)

conf_matrix_model1 = confusion_matrix(y_test_bc, y_pred_svm1)
conf_matrix_model2 = confusion_matrix(y_test_bc, y_pred_svm2)

fig, axs = plt.subplots(1, 2, figsize = (12, 4))

sns.heatmap(conf_matrix_model1, annot = True, cmap = 'Blues', ax = axs[0])
axs[0].set_title('Model 1')

sns.heatmap(conf_matrix_model2, annot = True, cmap = 'Blues', ax = axs[1])
axs[1].set_title('Model 2')

axs[0].set_xlabel('Predicted label')
axs[0].set_ylabel('True label')
axs[1].set_xlabel('Predicted label')
plt.show()

In [ ]:
y_prob_svm1 = svm1.predict_proba(X_test_bc)[:,1]
y_prob_svm2 = svm2.predict_proba(X_test_bc)[:,1]

fpr1, tpr1, _ = roc_curve(y_test_bc, y_prob_svm1)
roc_auc1 = auc(fpr1, tpr1)

fpr2, tpr2, _ = roc_curve(y_test_bc, y_prob_svm2)
roc_auc2 = auc(fpr2, tpr2)

fig, axes = plt.subplots(1, 3, figsize = (15, 5))

axes[0].plot(fpr1, tpr1, label = f'ROC curve (area = {roc_auc1:.2%})', color = colors[1])
axes[0].plot([0, 1], [0, 1], color = colors[0], linestyle = '--')
axes[0].set_xlim([-0.05, 1.0])
axes[0].set_ylim([0.0, 1.05])
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate')
axes[0].set_title('ROC Curve (Model 1)')
axes[0].legend(loc = 'lower right')

axes[1].plot(fpr2, tpr2, label = f'ROC curve (area = {roc_auc2:.2%})', color = colors[2])
axes[1].plot([0, 1], [0, 1], color = colors[0], linestyle = '--')
axes[1].set_xlim([-0.05, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('ROC Curve (Model 2)')
axes[1].legend(loc = 'lower right')

axes[2].plot(fpr1, tpr1, label = f'ROC curve (area = {roc_auc1:.2%})', color = colors[1])
axes[2].plot(fpr2, tpr2, label = f'ROC curve (area = {roc_auc2:.2%})', color = colors[2])
axes[2].plot([0, 1], [0, 1], color = colors[0], linestyle = '--')
axes[2].set_xlim([-0.05, 1.0])
axes[2].set_ylim([0.0, 1.05])
axes[2].set_xlabel('False Positive Rate')
axes[2].set_ylabel('True Positive Rate')
axes[2].set_title('Model 1 vs Model 2')
axes[2].legend(loc = 'lower right')

plt.tight_layout()
plt.show()

In [ ]:
precision1, recall1, threshold1 = precision_recall_curve(y_test_bc, y_prob_svm1)
precision2, recall2, threshold2 = precision_recall_curve(y_test_bc, y_prob_svm2)

fig, axs = plt.subplots(1, 3, figsize = (15, 5))

axs[0].plot(recall1, precision1, color = colors[1])
axs[0].set_xlabel('Recall')
axs[0].set_ylabel('Precision')
axs[0].set_title('Precision-Recall Curve (Model 1)')

axs[1].plot(recall2, precision2, color = colors[2])
axs[1].set_xlabel('Recall')
axs[1].set_ylabel('Precision')
axs[1].set_title('Precision-Recall Curve (Model 2)')

axs[2].plot(recall1, precision1, color = colors[1], label = 'Model 1')
axs[2].plot(recall2, precision2, color = colors[2], label = 'Model 2')
axs[2].set_xlabel('Recall')
axs[2].set_ylabel('Precision')
axs[2].set_title('Model 1 vs Model 2')
axs[2].legend(loc = 'lower left')

plt.tight_layout()
plt.show()

In [ ]:
target_names = svm1.classes_
metrics1 = classification_report(y_true = y_test_bc, y_pred = y_pred_svm1, target_names = target_names, output_dict = True)
precision1 = [metrics1[target_name]['precision'] for target_name in target_names]
recall1 = [metrics1[target_name]['recall'] for target_name in target_names]
f1_score1 = [metrics1[target_name]['f1-score'] for target_name in target_names]

metrics2 = classification_report(y_true = y_test_bc, y_pred = y_pred_svm2, target_names = target_names, output_dict = True)
precision2 = [metrics2[target_name]['precision'] for target_name in target_names]
recall2 = [metrics2[target_name]['recall'] for target_name in target_names]
f1_score2 = [metrics2[target_name]['f1-score'] for target_name in target_names]

data1 = np.array([precision1, recall1, f1_score1])
data2 = np.array([precision2, recall2, f1_score2])
rows = ['Precision', 'Recall', 'F1-score']

fig, axs = plt.subplots(1, 2, figsize=(14, 6))
sns.heatmap(data1, cmap = 'Pastel1', annot = True, fmt = '.2f', xticklabels = target_names, yticklabels = rows, ax = axs[0])
sns.heatmap(data2, cmap = 'Pastel1', annot = True, fmt = '.2f', xticklabels = target_names, yticklabels = rows, ax = axs[1])
axs[0].set_title('Classification Report (Model 1)')
axs[1].set_title('Classification Report (Model 2)')
plt.show()

In [ ]:
palette = sns.color_palette('Blues', n_colors = 2)

acc1 = accuracy_score(y_pred_svm1, y_test_bc)
acc2 = accuracy_score(y_pred_svm2, y_test_bc)

labels = ['Model 1', 'Model 2']
scores = [acc1, acc2]

fig, ax = plt.subplots(figsize = (9, 3))
ax.barh(labels, scores, color = palette)
ax.set_xlim([0, 1])
ax.set_xlabel('Accuracy Score')
ax.set_title('Support Vector Machine Model Comparison')

for i, v in enumerate(scores):
    ax.text(v + 0.01, i, str(round(v, 3)), ha = 'left', va = 'center')

plt.show()

In [ ]:
palette = sns.color_palette('Greens', n_colors = 2)

labels = ['Model 1', 'Model 2']
scores = [cv_svm1.mean(), cv_svm2.mean()]

fig, ax = plt.subplots(figsize = (9, 3))
ax.barh(labels, scores, color = palette)
ax.set_xlim([0, 1])
ax.set_xlabel('Cross Validation Score')
ax.set_title('Support Vector Machine Model Comparison (Cross Validation)')

for i, v in enumerate(scores):
    ax.text(v + 0.01, i, str(round(v, 3)), ha = 'left', va = 'center')

plt.show()

## Comparação dos algoritmos de classificação binária

Treinámos dois modelos para cada algoritmo de classificação diferente. Para comparar os diferentes algoritmos, seleccionaremos o modelo com melhor desempenho de cada classe com base na precisão, recuperação, exatidão, etc. do modelo.

1. Regressão logística: Modelo 2
2. Máquina de vetor de suporte: Modelo 2

In [ ]:
conf_matrix_model1 = confusion_matrix(y_test_bc, y_pred_lr2)
conf_matrix_model2 = confusion_matrix(y_test_bc, y_pred_svm2)

fig, axs = plt.subplots(1, 2, figsize = (12, 4))

sns.heatmap(conf_matrix_model1, annot = True, cmap = 'Blues', ax = axs[0])
axs[0].set_title('Logistic Regression')

sns.heatmap(conf_matrix_model2, annot = True, cmap = 'Blues', ax = axs[1])
axs[1].set_title('Support Vector Machine')

axs[0].set_xlabel('Predicted label')
axs[0].set_ylabel('True label')
axs[1].set_xlabel('Predicted label')
plt.show()

In [ ]:
fpr1, tpr1, _ = roc_curve(y_test_bc, y_prob_lr2)
roc_auc1 = auc(fpr1, tpr1)

fpr2, tpr2, _ = roc_curve(y_test_bc, y_prob_svm2)
roc_auc2 = auc(fpr2, tpr2)

fig, axes = plt.subplots(1, 3, figsize = (15, 5))

axes[0].plot(fpr1, tpr1, label = f'ROC curve (area = {roc_auc1:.2%})', color = colors[1])
axes[0].plot([0, 1], [0, 1], color = colors[0], linestyle = '--')
axes[0].set_xlim([-0.05, 1.0])
axes[0].set_ylim([0.0, 1.05])
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate')
axes[0].set_title('ROC Curve (Logistic Regression)')
axes[0].legend(loc = 'lower right')

axes[1].plot(fpr2, tpr2, label = f'ROC curve (area = {roc_auc2:.2%})', color = colors[2])
axes[1].plot([0, 1], [0, 1], color = colors[0], linestyle = '--')
axes[1].set_xlim([-0.05, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('ROC Curve (SVM)')
axes[1].legend(loc = 'lower right')

axes[2].plot(fpr1, tpr1, label = f'LR ROC curve (area = {roc_auc1:.2%})', color = colors[1])
axes[2].plot(fpr2, tpr2, label = f'SVM ROC curve (area = {roc_auc2:.2%})', color = colors[2])
axes[2].plot([0, 1], [0, 1], color = colors[0], linestyle = '--')
axes[2].set_xlim([-0.05, 1.0])
axes[2].set_ylim([0.0, 1.05])
axes[2].set_xlabel('False Positive Rate')
axes[2].set_ylabel('True Positive Rate')
axes[2].set_title('LR vs SVM')
axes[2].legend(loc = 'lower right')

plt.tight_layout()
plt.show()

In [ ]:
precision1, recall1, threshold1 = precision_recall_curve(y_test_bc, y_prob_lr2)
precision2, recall2, threshold2 = precision_recall_curve(y_test_bc, y_prob_svm2)

fig, axs = plt.subplots(1, 3, figsize = (15, 5))

axs[0].plot(recall1, precision1, color = colors[1], label = 'Model 1')
axs[0].set_xlabel('Recall')
axs[0].set_ylabel('Precision')
axs[0].set_title('Precision-Recall Curve (LR)')

axs[1].plot(recall2, precision2, color = colors[2], label = 'Model 2')
axs[1].set_xlabel('Recall')
axs[1].set_ylabel('Precision')
axs[1].set_title('Precision-Recall Curve (SVM)')

axs[2].plot(recall1, precision1, color = colors[1], label = 'Logistic Regression')
axs[2].plot(recall2, precision2, color = colors[2], label = 'Support Vector Machine')
axs[2].set_xlabel('Recall')
axs[2].set_ylabel('Precision')
axs[2].set_title('LR vs SVM')
axs[2].legend(loc = 'lower left')

plt.tight_layout()
plt.show()

In [ ]:
target_names = svm2.classes_
metrics1 = classification_report(y_true = y_test_bc, y_pred = y_pred_lr2, target_names = target_names, output_dict = True)
precision1 = [metrics1[target_name]['precision'] for target_name in target_names]
recall1 = [metrics1[target_name]['recall'] for target_name in target_names]
f1_score1 = [metrics1[target_name]['f1-score'] for target_name in target_names]

metrics2 = classification_report(y_true = y_test_bc, y_pred = y_pred_svm2, target_names = target_names, output_dict = True)
precision2 = [metrics2[target_name]['precision'] for target_name in target_names]
recall2 = [metrics2[target_name]['recall'] for target_name in target_names]
f1_score2 = [metrics2[target_name]['f1-score'] for target_name in target_names]

data1 = np.array([precision1, recall1, f1_score1])
data2 = np.array([precision2, recall2, f1_score2])
rows = ['Precision', 'Recall', 'F1-score']

fig, axs = plt.subplots(1, 2, figsize = (14, 6))
sns.heatmap(data1, cmap = 'Pastel1', annot = True, fmt = '.2f', xticklabels = target_names, yticklabels = rows, ax=axs[0])
sns.heatmap(data2, cmap = 'Pastel1', annot = True, fmt = '.2f', xticklabels = target_names, yticklabels = rows, ax=axs[1])
axs[0].set_title('Classification Report (LR)')
axs[1].set_title('Classification Report (SVM)')
plt.show()

In [ ]:
palette = sns.color_palette('Blues', n_colors = 2)

acc1 = accuracy_score(y_pred_lr2, y_test_bc)
acc2 = accuracy_score(y_pred_svm2, y_test_bc)

labels = ['Logistic Regression', 'Support Vector Machine']
scores = [acc1, acc2]

fig, ax = plt.subplots(figsize = (9, 3))
ax.barh(labels, scores, color = palette)
ax.set_xlim([0, 1])
ax.set_xlabel('Accuracy Score')
ax.set_title('Binary Classification Model Comparison')

for i, v in enumerate(scores):
    ax.text(v + 0.01, i, str(round(v, 3)), ha = 'left', va = 'center')

plt.show()

In [ ]:
palette = sns.color_palette('Greens', n_colors = 2)

labels = ['Logistic Regression', 'Support Vector Machine']
scores = [cv_lr2.mean(), cv_svm2.mean()]

fig, ax = plt.subplots(figsize = (9, 3))
ax.barh(labels, scores, color = palette)
ax.set_xlim([0, 1])
ax.set_xlabel('Cross Validation Score')
ax.set_title('Binary Classification Model Comparison')

for i, v in enumerate(scores):
    ax.text(v + 0.01, i, str(round(v, 3)), ha = 'left', va = 'center')

plt.show()

### Random Forest Models Comparison

In [ ]:
y_pred_rf1 = rf1.predict(X_test)
y_pred_rf2 = rf2.predict(X_test)

conf_matrix_model1 = confusion_matrix(y_test, y_pred_rf1)
conf_matrix_model2 = confusion_matrix(y_test, y_pred_rf2)

fig, axs = plt.subplots(1, 2, figsize = (16, 7))

sns.heatmap(conf_matrix_model1, annot = True, cmap = 'Blues', ax = axs[0], xticklabels = rf1.classes_, yticklabels = rf1.classes_)
axs[0].set_title('Model 1')

sns.heatmap(conf_matrix_model2, annot = True, cmap = 'Blues', ax = axs[1], xticklabels = rf2.classes_, yticklabels = rf2.classes_)
axs[1].set_title('Model 2')

axs[0].set_xlabel('Predicted label')
axs[0].set_ylabel('True label')
axs[1].set_xlabel('Predicted label')

fig.tight_layout()
plt.show()

In [ ]:
target_names = rf1.classes_
metrics1 = classification_report(y_true = y_test, y_pred = y_pred_rf1, target_names = target_names, output_dict = True)
precision1 = [metrics1[target_name]['precision'] for target_name in target_names]
recall1 = [metrics1[target_name]['recall'] for target_name in target_names]
f1_score1 = [metrics1[target_name]['f1-score'] for target_name in target_names]

metrics2 = classification_report(y_true = y_test, y_pred = y_pred_rf2, target_names = target_names, output_dict = True)
precision2 = [metrics2[target_name]['precision'] for target_name in target_names]
recall2 = [metrics2[target_name]['recall'] for target_name in target_names]
f1_score2 = [metrics2[target_name]['f1-score'] for target_name in target_names]

data1 = np.array([precision1, recall1, f1_score1])
data2 = np.array([precision2, recall2, f1_score2])
rows = ['Precision', 'Recall', 'F1-score']

fig, axs = plt.subplots(1, 2, figsize = (14, 6))
sns.heatmap(data1, cmap = 'Pastel1', annot = True, fmt = '.2f', xticklabels = target_names, yticklabels = rows, ax = axs[0])
sns.heatmap(data2, cmap = 'Pastel1', annot = True, fmt = '.2f', xticklabels = target_names, yticklabels = rows, ax = axs[1])
axs[0].set_title('Classification Report (Model 1)')
axs[1].set_title('Classification Report (Model 2)')
fig.tight_layout()
plt.show()

In [ ]:
palette = sns.color_palette('Blues', n_colors = 2)

acc1 = accuracy_score(y_pred_rf1, y_test)
acc2 = accuracy_score(y_pred_rf2, y_test)

labels = ['Model 1', 'Model 2']
scores = [acc1, acc2]

fig, ax = plt.subplots(figsize = (9, 3))
ax.barh(labels, scores, color = palette)
ax.set_xlim([0, 1])
ax.set_xlabel('Accuracy Score')
ax.set_title('Random Forest Model Comparison')

for i, v in enumerate(scores):
    ax.text(v + 0.01, i, str(round(v, 3)), ha = 'left', va = 'center')

plt.show()

In [ ]:
palette = sns.color_palette('Greens', n_colors = 2)

labels = ['Model 1', 'Model 2']
scores = [cv_rf1.mean(), cv_rf2.mean()]

fig, ax = plt.subplots(figsize = (9, 3))
ax.barh(labels, scores, color = palette)
ax.set_xlim([0, 1])
ax.set_xlabel('Cross Validation Score')
ax.set_title('Support Vector Machine Model Comparison (Cross Validation)')

for i, v in enumerate(scores):
    ax.text(v + 0.01, i, str(round(v, 3)), ha = 'left', va = 'center')

plt.show()

### Decision Trees Models Comparison

In [ ]:
y_pred_dt1 = dt1.predict(X_test)
y_pred_dt2 = dt2.predict(X_test)

conf_matrix_model1 = confusion_matrix(y_test, y_pred_dt1)
conf_matrix_model2 = confusion_matrix(y_test, y_pred_dt2)

fig, axs = plt.subplots(1, 2, figsize = (16, 7))

sns.heatmap(conf_matrix_model1, annot = True, cmap = 'Blues', ax = axs[0], xticklabels = dt1.classes_, yticklabels = dt1.classes_)
axs[0].set_title('Model 1')

sns.heatmap(conf_matrix_model2, annot = True, cmap = 'Blues', ax = axs[1], xticklabels = dt2.classes_, yticklabels = dt2.classes_)
axs[1].set_title('Model 2')

axs[0].set_xlabel('Predicted label')
axs[0].set_ylabel('True label')
axs[1].set_xlabel('Predicted label')

fig.tight_layout()
plt.show()

In [ ]:
target_names = dt1.classes_
metrics1 = classification_report(y_true = y_test, y_pred = y_pred_dt1, target_names = target_names, output_dict = True)
precision1 = [metrics1[target_name]['precision'] for target_name in target_names]
recall1 = [metrics1[target_name]['recall'] for target_name in target_names]
f1_score1 = [metrics1[target_name]['f1-score'] for target_name in target_names]

metrics2 = classification_report(y_true = y_test, y_pred = y_pred_dt2, target_names = target_names, output_dict = True)
precision2 = [metrics2[target_name]['precision'] for target_name in target_names]
recall2 = [metrics2[target_name]['recall'] for target_name in target_names]
f1_score2 = [metrics2[target_name]['f1-score'] for target_name in target_names]

data1 = np.array([precision1, recall1, f1_score1])
data2 = np.array([precision2, recall2, f1_score2])
rows = ['Precision', 'Recall', 'F1-score']

fig, axs = plt.subplots(1, 2, figsize = (14, 6))
sns.heatmap(data1, cmap = 'Pastel1', annot = True, fmt = '.2f', xticklabels = target_names, yticklabels = rows, ax = axs[0])
sns.heatmap(data2, cmap = 'Pastel1', annot = True, fmt = '.2f', xticklabels = target_names, yticklabels = rows, ax = axs[1])
axs[0].set_title('Classification Report (Model 1)')
axs[1].set_title('Classification Report (Model 2)')
fig.tight_layout()
plt.show()

In [ ]:
palette = sns.color_palette('Blues', n_colors = 2)

acc1 = accuracy_score(y_pred_dt1, y_test)
acc2 = accuracy_score(y_pred_dt2, y_test)

labels = ['Model 1', 'Model 2']
scores = [acc1, acc2]

fig, ax = plt.subplots(figsize = (9, 3))
ax.barh(labels, scores, color = palette)
ax.set_xlim([0, 1])
ax.set_xlabel('Accuracy Score')
ax.set_title('Decision Trees Model Comparison')

for i, v in enumerate(scores):
    ax.text(v + 0.01, i, str(round(v, 3)), ha = 'left', va = 'center')

plt.show()

In [ ]:
palette = sns.color_palette('Greens', n_colors = 2)

labels = ['Model 1', 'Model 2']
scores = [cv_dt1.mean(), cv_dt2.mean()]

fig, ax = plt.subplots(figsize = (9, 3))
ax.barh(labels, scores, color = palette)
ax.set_xlim([0, 1])
ax.set_xlabel('Cross Validation Score')
ax.set_title('Decision Trees Model Comparison (Cross Validation)')

for i, v in enumerate(scores):
    ax.text(v + 0.01, i, str(round(v, 3)), ha = 'left', va = 'center')

plt.show()

### K Nearest Neighbours Models Comparison

In [ ]:
y_pred_knn1 = knn1.predict(X_test)
y_pred_knn2 = knn2.predict(X_test)

conf_matrix_model1 = confusion_matrix(y_test, y_pred_knn1)
conf_matrix_model2 = confusion_matrix(y_test, y_pred_knn2)

fig, axs = plt.subplots(1, 2, figsize = (16, 7))

sns.heatmap(conf_matrix_model1, annot = True, cmap = 'Blues', ax = axs[0], xticklabels = knn1.classes_, yticklabels = knn1.classes_)
axs[0].set_title('Model 1')

sns.heatmap(conf_matrix_model2, annot = True, cmap = 'Blues', ax = axs[1], xticklabels = knn2.classes_, yticklabels = knn2.classes_)
axs[1].set_title('Model 2')

axs[0].set_xlabel('Predicted label')
axs[0].set_ylabel('True label')
axs[1].set_xlabel('Predicted label')

fig.tight_layout()
plt.show()

In [ ]:
target_names = knn1.classes_
metrics1 = classification_report(y_true = y_test, y_pred = y_pred_knn1, target_names = target_names, output_dict = True)
precision1 = [metrics1[target_name]['precision'] for target_name in target_names]
recall1 = [metrics1[target_name]['recall'] for target_name in target_names]
f1_score1 = [metrics1[target_name]['f1-score'] for target_name in target_names]

metrics2 = classification_report(y_true = y_test, y_pred = y_pred_knn2, target_names = target_names, output_dict = True)
precision2 = [metrics2[target_name]['precision'] for target_name in target_names]
recall2 = [metrics2[target_name]['recall'] for target_name in target_names]
f1_score2 = [metrics2[target_name]['f1-score'] for target_name in target_names]

data1 = np.array([precision1, recall1, f1_score1])
data2 = np.array([precision2, recall2, f1_score2])
rows = ['Precision', 'Recall', 'F1-score']

fig, axs = plt.subplots(1, 2, figsize = (14, 6))
sns.heatmap(data1, cmap = 'Pastel1', annot = True, fmt = '.2f', xticklabels = target_names, yticklabels = rows, ax = axs[0])
sns.heatmap(data2, cmap = 'Pastel1', annot = True, fmt = '.2f', xticklabels = target_names, yticklabels = rows, ax = axs[1])
axs[0].set_title('Classification Report (Model 1)')
axs[1].set_title('Classification Report (Model 2)')
fig.tight_layout()
plt.show()

In [ ]:
palette = sns.color_palette('Blues', n_colors = 2)

acc1 = accuracy_score(y_pred_knn1, y_test)
acc2 = accuracy_score(y_pred_knn2, y_test)

labels = ['Model 1', 'Model 2']
scores = [acc1, acc2]

fig, ax = plt.subplots(figsize = (9, 3))
ax.barh(labels, scores, color = palette)
ax.set_xlim([0, 1])
ax.set_xlabel('Accuracy Score')
ax.set_title('K Nearest Neighbour Model Comparison')

for i, v in enumerate(scores):
    ax.text(v + 0.01, i, str(round(v, 3)), ha = 'left', va = 'center')

plt.show()

In [ ]:
palette = sns.color_palette('Greens', n_colors = 2)

labels = ['Model 1', 'Model 2']
scores = [cv_knn1.mean(), cv_knn2.mean()]

fig, ax = plt.subplots(figsize = (9, 3))
ax.barh(labels, scores, color = palette)
ax.set_xlim([0, 1])
ax.set_xlabel('Cross Validation Score')
ax.set_title('Decision Trees Model Comparison (Cross Validation)')

for i, v in enumerate(scores):
    ax.text(v + 0.01, i, str(round(v, 3)), ha = 'left', va = 'center')

plt.show()

### Comparação dos algoritmos de classificação multi-classe

Treinámos dois modelos para cada algoritmo de classificação diferente. Para comparar os diferentes algoritmos, seleccionaremos o modelo com melhor desempenho de cada classe, com base na precisão, recuperação, exatidão, etc. do modelo.

Floresta aleatória: Modelo 2
Árvores de decisão: Modelo 2
KNN: Modelo 2

In [ ]:
palette = sns.color_palette('Blues', n_colors = 3)

rf_acc = accuracy_score(y_pred_rf2, y_test)
dt_acc = accuracy_score(y_pred_dt2, y_test)
knn_acc = accuracy_score(y_pred_knn2, y_test)

labels = ['Random Forest', 'Decision Trees', 'K Nearest Neighbours']
scores = [rf_acc, dt_acc, knn_acc]

fig, ax = plt.subplots(figsize = (9, 3))
ax.barh(labels, scores, color = palette)
ax.set_xlim([0, 1])
ax.set_xlabel('Accuracy Score')
ax.set_title('Multi-class Classification Model Comparison')

for i, v in enumerate(scores):
    ax.text(v + 0.01, i, str(round(v, 4)), ha = 'left', va = 'center')

plt.show()

In [ ]:
palette = sns.color_palette('Greens', n_colors = 3)

labels = ['Random Forest', 'Decision Trees', 'K Nearest Neighbours']
scores = [cv_rf2.mean(), cv_dt2.mean(), cv_knn2.mean()]

fig, ax = plt.subplots(figsize = (9, 3))
ax.barh(labels, scores, color = palette)
ax.set_xlim([0, 1])
ax.set_xlabel('Cross Validation Score')
ax.set_title('Multi-class Classification Model Comparison')

for i, v in enumerate(scores):
    ax.text(v + 0.01, i, str(round(v, 4)), ha = 'left', va = 'center')

plt.show()

In [ ]:
target_names = rf2.classes_
preds = [y_pred_rf2, y_pred_dt2, y_pred_knn2]

datas = []
for pred in preds:
    metrics = classification_report(y_true = y_test, y_pred = pred, target_names = target_names, output_dict = True)
    precision = [metrics[target_name]['precision'] for target_name in target_names]
    recall = [metrics[target_name]['recall'] for target_name in target_names]
    f1_score = [metrics[target_name]['f1-score'] for target_name in target_names]

    datas.append(np.array([precision, recall, f1_score]))

rows = ['Precision', 'Recall', 'F1-score']

fig, axs = plt.subplots(1, 3, figsize = (19, 6))
sns.heatmap(datas[0], cmap = 'Pastel1', annot = True, fmt = '.2f', xticklabels = target_names, yticklabels = rows, ax = axs[0])
sns.heatmap(datas[1], cmap = 'Pastel1', annot = True, fmt = '.2f', xticklabels = target_names, yticklabels = rows, ax = axs[1])
sns.heatmap(datas[2], cmap = 'Pastel1', annot = True, fmt = '.2f', xticklabels = target_names, yticklabels = rows, ax = axs[2])

axs[0].set_title('Classification Report (Random Forest)')
axs[1].set_title('Classification Report (Decision Trees)')
axs[2].set_title('Classification Report (K Nearest Neighbours)')
fig.tight_layout()
plt.show()

In [ ]:
preds = [y_pred_rf2, y_pred_dt2, y_pred_knn2]

conf_matrix = [confusion_matrix(y_test, y_pred) for y_pred in preds]

fig, axs = plt.subplots(1, 3, figsize = (22, 8))

sns.heatmap(conf_matrix[0], annot = True, cmap = 'Blues', ax = axs[0], xticklabels = dt1.classes_, yticklabels = dt1.classes_)
sns.heatmap(conf_matrix[1], annot = True, cmap = 'Blues', ax = axs[1], xticklabels = dt1.classes_, yticklabels = dt1.classes_)
sns.heatmap(conf_matrix[2], annot = True, cmap = 'Blues', ax = axs[2], xticklabels = dt1.classes_, yticklabels = dt1.classes_)

axs[0].set_title('Confusion Matrix (Random Forest)')
axs[1].set_title('Confusion Matrix (Decision Trees)')
axs[2].set_title('Confusion Matrix (K Nearest Neighbours)')

axs[0].set_xlabel('Predicted label')
axs[1].set_xlabel('Predicted label')
axs[2].set_xlabel('Predicted label')
axs[0].set_ylabel('True label')

fig.tight_layout()
plt.show()

Durante a fase de treino de vários algoritmos de classificação, treinámos dois modelos para cada um dos algoritmos utilizando parâmetros diferentes.

***Algoritmos de classificação binária: ***
1. Regressão logística
2. Máquina de vectores de apoio

Para comparar os diferentes algoritmos de classificação, utilizámos os modelos com melhor desempenho de cada algoritmo.

Com base nos nossos vários testes, descobrimos que a regressão logística pode lidar com dados de grande dimensão e pode ser treinada num período de tempo muito curto. No entanto, a desvantagem são os modelos menos exactos. Por outro lado, o SVM é bastante dispendioso do ponto de vista computacional e demora muito tempo a ser treinado, mas o aspeto positivo é que a pontuação de precisão é muito superior à dos modelos de regressão logística. Ajustámos alguns parâmetros aqui e ali para melhorar a precisão dos modelos. Também fizemos uma validação cruzada para garantir que o nosso modelo não estava sobreajustado e que tinha sido treinado corretamente.

Um outro aspeto a mencionar é que a precisão dos modelos depende da normalização do conjunto de dados. Treinámos o nosso modelo com e sem a utilização de um scaler padrão. Durante o teste de diferentes modelos, descobrimos que a precisão e outras pontuações de medidas de desempenho aumentaram significativamente após a normalização do conjunto de dados. Por isso, optámos por normalizar os dados antes de aplicar o PCA.

***Algoritmos de classificação multi-classe: ***
1. Floresta aleatória
2. Árvores de decisão
3. K Vizinhos mais próximos

Mais uma vez, para comparar os diferentes algoritmos de classificação, utilizámos os modelos com melhor desempenho de cada um dos 3 algoritmos.

O tempo necessário para treinar os modelos de classificação multi-classe é relativamente inferior ao dos modelos de classificação binária. Isto pode dever-se ao facto de a dimensão dos dados de treino ser inferior à dos dados de treino da classificação binária.

Tal como indicado na secção de análise, o conjunto de dados é altamente desequilibrado. A fim de manter os dados equilibrados em todas as classes, começámos por recolher o maior número de amostras das classes minoritárias e um número suficiente de amostras das classes maioritárias. Mais tarde, utilizámos a técnica SMOTE (Synthetic Minority Over-sampling Technique) para criar um conjunto de dados globalmente equilibrado para treinar os modelos de classificação multiclasse.

Ao comparar as métricas de desempenho dos modelos, verificamos que a Floresta Aleatória é o modelo com melhor desempenho, seguido do KNN e da Árvore de Decisão. A partir da matriz de confusão e do relatório de classificação, é evidente o domínio da Floresta Aleatória em termos de precisão, recuperação e pontuação f1. A razão pela qual a Árvore de Decisão fica para trás é o facto de nem sempre ser suficientemente expressiva para captar relações complexas entre as caraterísticas de entrada e a variável-alvo. A árvore de decisão pode ter dificuldades com problemas em que a variável-alvo depende de uma combinação de caraterísticas de entrada em vez de apenas uma ou duas caraterísticas. O KNN e o Random Forest podem lidar com relações mais complexas entre as caraterísticas de entrada e a variável-alvo, utilizando modelos mais flexíveis. Além disso, utilizámos relativamente menos parâmetros para ajustar os modelos de árvore de decisão. Tal como os algoritmos de classificação binária, também efectuámos uma validação cruzada dos modelos de classificação multi-classe para garantir que não estavam demasiado ajustados.

**Trabalho futuro:** Tendo isto em mente, temos de escolher o nosso modelo em conformidade se quisermos ficar com um só. No entanto, neste caso, também podemos combinar os classificadores KNN e Random Forest utilizando um método de conjunto. Isto pode melhorar a precisão do nosso sistema de deteção de intrusões, aproveitando os pontos fortes de ambos os modelos e reduzindo o risco de sobreajuste. O método de conjunto permitiria que os modelos trabalhassem em conjunto para produzir uma previsão mais robusta, que poderia ser mais eficaz na identificação de diferentes tipos de ataque à rede.

# 6. Salvar o Modelo 

In [ ]:
import joblib  # Biblioteca para salvar modelos
import os

# nn_model = Rede Neural treinada
# svm_model = SVM treinado

# Diretório onde os modelos serão salvos
output_dir = "modelos_salvos"
os.makedirs(output_dir, exist_ok=True)

# Salvando o modelo da Rede Neural
#joblib.dump(model, os.path.join(output_dir, 'neural_network_model.pkl'))

# Salvando o modelo SVM (Máquina de Vetores de Suporte)
joblib.dump(svm2, os.path.join(output_dir, 'svm_model_impl.pkl'))

print("Modelos salvos com sucesso!")